In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import socket
from urllib3.connection import HTTPConnection
import matplotlib.pyplot as plt
import seaborn as sns
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [3]:
HTTPConnection.default_socket_options = (
    HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1),
        (socket.SOL_TCP, socket.TCP_KEEPIDLE, 45),
        (socket.SOL_TCP, socket.TCP_KEEPINTVL, 10),
        (socket.SOL_TCP, socket.TCP_KEEPCNT, 6)
    ] )

In [31]:
len(descripcion)

20

In [18]:
tipomoneda_vivienda=[]
precio_vivienda=[]
direccion_vivienda=[]
barrio_vivienda=[]
#metros_cuadrados_vivienda=[]
#años_vivienda=[]
#ambientes_vivienda=[]
descripcion_viv=[]

link_inmobiliaria=[]



for i in range(0,157):
    
    if i==0:
        link = 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'
        
    else:
        link= 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'+str("-pagina-{}".format(i))
        


    driver = webdriver.Chrome()



    driver.get(link)

    time.sleep(15)

    html = driver.page_source

    driver.implicitly_wait(20)

    soup= BeautifulSoup(html,'html.parser')

    seleccionador_precio =soup.find_all('p',class_='card__price')

    tipo_moneda=[]
    precio=[]
    
    for currency in seleccionador_precio:
        try:
            #SELECCION DE TIPO DE MONEDA
            #SELECCION DE PRECIO
            moneda=currency.find('span',class_='card__currency').text
            tipo_moneda.append(moneda)
            valor=currency.text
            precio.append(valor)
        except:
            tipo_moneda.append('faltante')
        
            precio.append('faltante')

    #PRIMER LOOP PARA ENCONTRAR EL TAG
    #SEGUNDO LOOP PARA AGARRAR EL SRC
    #
    img_l = soup.find_all('div',class_='card__agent')

    link=[]

    for i in img_l:
        if i.img== None:
            link.append('no hay')
        else:

            img_ll=i.img
            link.append(img_ll)

    link_src=[]
    for src in link:
        if src== 'no hay':
            link_src.append('No aparece Inmobiliaria')
        else:
        
            link_src.append(src['src'])

    ################################################################3
    
    seleccionador_ubicacion = soup.find_all('div',class_='card__monetary-values')
    direccion=[]
    dir_barrio=[]
    for i in seleccionador_ubicacion:
        try:
            lugar=i.find('h2').text
            direccion.append(lugar)

            barrio=i.select('p')[1].text
            dir_barrio.append(barrio)
        except:
            direccion.append('falta lugar')
            dir_barrio.append('falta barrio')

     ######################################################################
            
    #metros_cuadrados =[]
    #años=[]
    #ambientes=[]
    descripcion=[]
    uls = soup.find_all('ul',class_='card__main-features')

    for enum,ul in enumerate(uls):
        
        spans=ul.find_all('span')
        for span in spans:
            
            text=span.text
            descripcion.append(f'{enum}:{text}')
        

    
    tipomoneda_vivienda.append(tipo_moneda)
    precio_vivienda.append(precio)
    direccion_vivienda.append(direccion)
    barrio_vivienda.append(dir_barrio)
    #metros_cuadrados_vivienda.append(metros_cuadrados)
    #años_vivienda.append(años)
    #ambientes_vivienda.append(ambientes)
    link_inmobiliaria.append(link_src)
    descripcion_viv.append(descripcion)
    
    driver.quit()

    
    

In [19]:
print(len(tipomoneda_vivienda))
print(len(precio_vivienda))
print(len(direccion_vivienda))
print(len(barrio_vivienda))

print(len(descripcion_viv))
print(len(link_inmobiliaria))

157
157
157
157
157
157


106

In [20]:
df =pd.DataFrame({'tipo_moneda':tipomoneda_vivienda,'precio_vivienda':precio_vivienda,
                'direccion_vivienda':direccion_vivienda,'barrio_vivienda':barrio_vivienda,
                'descripcion_vivenda':descripcion_viv,'inmobiliaria':link_inmobiliaria})

In [21]:
df

,tipo_moneda,precio_vivienda,direccion_vivienda,barrio_vivienda,descripcion_vivenda,inmobiliaria
0,"[USD, USD, USD, USD, USD, $, $, $, $, $, $, $,...",[\nUSD 1.500 \n ...,[\n Juan Maria Gutierrez 39...,"[Departamento en Alquiler en Palermo, Capital ...",[0:\n 2 dorm.\n ...,[https://static1.sosiva451.com/452241_a/5b0a03...
1,"[USD, USD, USD, USD, USD, $, $, $, $, $, $, $,...",[\nUSD 1.500 \n ...,[\n Juan Maria Gutierrez 39...,"[Departamento en Alquiler en Palermo, Capital ...",[0:\n 2 dorm.\n ...,[https://static1.sosiva451.com/452241_a/5b0a03...
2,"[$, $, $, $, USD, USD, $, $, $, $, $, $, $, $,...",[\n$ 390.000 \n ...,[\n La Pampa al 2000\n ...,"[Departamento en Alquiler en Belgrano, Capital...",[0:\n 150 m² cubie.\n...,[https://static1.sosiva451.com/347151_a/527a2f...
3,"[$, $, $, $, $, $, $, $, $, USD, $, USD, $, $,...",[\n$ 90.000 \n \...,"[\n Av Diaz Velez 4500, Pis...","[Departamento en Alquiler en Caballito, Capita...",[0:\n 37 m² cubie.\n ...,"[No aparece Inmobiliaria, https://static1.sosi..."
4,"[$, USD, $, $, $, $, $, USD, $, $, $, $, $, $,...","[\n$ 140.000 \n , \n...",[\n Arenales 1100\n ...,"[Departamento en Alquiler en Recoleta, Barrio ...",[0:\n 30 m² cubie.\n ...,"[No aparece Inmobiliaria, https://static1.sosi..."
...,...,...,...,...,...,...
152,"[$, $, $, $, $, $, $, $, $, $, $, $, USD, USD,...",[\n$ 165.000 \n ...,[\n LA PAMPA 4200\n ...,"[Departamento en Alquiler en Belgrano R, Belgr...",[0:\n 47 m² cubie.\n ...,[https://static1.sosiva451.com/200201_a/4e915e...
153,"[$, $, $, $, $, $, USD, $, $, USD, USD, $, $, ...",[\n$ 80.000 \n \...,[\n TRIUNVIRATO AV. 4100\n ...,"[Departamento en Alquiler en Villa Urquiza, Ca...",[0:\n 36 m² cubie.\n ...,[https://static1.sosiva451.com/454101_a/c4d54f...
154,"[USD, USD, $, USD, $, $, $, $, $, $, $, $, $, ...","[\nUSD 900 \n , \nUS...",[\n Soler 6000\n ...,[Departamento en Alquiler en Palermo Hollywood...,[0:\n 92 m² cubie.\n ...,[https://static1.sosiva451.com/530752_a/bb44e5...
155,"[$, USD, USD, $, $, $, USD, $, $, USD, USD, $,...",[\n$ 68.000 \n \...,[\n Junin al 700\n ...,"[Departamento en Alquiler en Balvanera, Capita...",[0:\n 32 m² cubie.\n ...,[https://static1.sosiva451.com/843762_a/18ad69...


In [87]:
df.to_csv('argenprop.csv',index=False)

In [16]:
tipomoneda_vivienda=[]
precio_vivienda=[]
direccion_vivienda=[]
barrio_vivienda=[]
#metros_cuadrados_vivienda=[]
#años_vivienda=[]
#ambientes_vivienda=[]
descripcion_viv=[]

link_inmobiliaria=[]


lista=[1,2]

for i in lista:
    
    if i==0:
        link = 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'
        
    else:
        link= 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'+str("-pagina-{}".format(i))
        


    driver = webdriver.Chrome()



    driver.get(link)

    time.sleep(15)

    html = driver.page_source

    driver.implicitly_wait(20)

    soup= BeautifulSoup(html,'html.parser')

    #seleccionador_precio =soup.find_all('p',class_='card__price')

    #tipo_moneda=[]
    #precio=[]
    
   # for currency in seleccionador_precio:
       # try:
            #SELECCION DE TIPO DE MONEDA
            #SELECCION DE PRECIO
          #  moneda=currency.find('span',class_='card__currency').text
           # tipo_moneda.append(moneda)
          #  valor=currency.text
           # precio.append(valor)
      #  except:
          #  tipo_moneda.append('faltante')
        
           # precio.append('faltante')

    #PRIMER LOOP PARA ENCONTRAR EL TAG
    #SEGUNDO LOOP PARA AGARRAR EL SRC
    #
   # img_l = soup.find_all('div',class_='card__agent')

  #  link=[]

  #  for i in img_l:
   #     if i.img== None:
    #        link.append('no hay')
    #    else:

      #      img_ll=i.img
       #     link.append(img_ll)

 #   link_src=[]
  #  for src in link:
    #    if src== 'no hay':
     #       link_src.append('No aparece Inmobiliaria')
      #  else:
        
        #    link_src.append(src['src'])

    ################################################################3
    
    #seleccionador_ubicacion = soup.find_all('div',class_='card__monetary-values')
    #direccion=[]
    #dir_barrio=[]
    #for i in seleccionador_ubicacion:
    #    try:
    #        lugar=i.find('h2').text
     #       direccion.append(lugar)

      #      barrio=i.select('p')[1].text
      #      dir_barrio.append(barrio)
       # except:
      #      direccion.append('falta lugar')
      #      dir_barrio.append('falta barrio')

     ######################################################################
            
    #metros_cuadrados =[]
    #años=[]
    #ambientes=[]
    descripcion=[]
    uls = soup.find_all('ul',class_='card__main-features')

    for enum,ul in enumerate(uls):
        #a=precision.find_all('li')
        spans=ul.find_all('span')
        for span in spans:
            print(f'{enum}:{span.text}')
            text=span.text
            descripcion.append(f'{enum}:{text}')
            
        #descripcion_span= [
        
        #for i in a:
         #   b=i.find('span').text
          #  print(b)
            
            
         #   descripcion_span.append(b)

                
                
                
        #descripcion.append(descripcion_span)

    
    #tipomoneda_vivienda.append(tipo_moneda)
    #precio_vivienda.append(precio)
    #direccion_vivienda.append(direccion)
    #barrio_vivienda.append(dir_barrio)
    #metros_cuadrados_vivienda.append(metros_cuadrados)
    #años_vivienda.append(años)
    #ambientes_vivienda.append(ambientes)
    #link_inmobiliaria.append(link_src)
    descripcion_viv.append(descripcion)
    
    driver.quit()

    
    

0:
                         2 dorm.
                    
0:
                         15 años
                    
0:
                         2 baños
                    
0:
                         3 ambientes
                    
0:
                         1 cocheras
                    
0:
                         Excelente
                    
0:
                         Contra.
                    
1:
                         400  m² cubie.
                    
1:
                         5 dorm.
                    
1:
                         30 años
                    
1:
                         3 baños
                    
1:
                         4 ambientes
                    
1:
                         3 cocheras
                    
1:
                         1 toilettes
                    
1:
                         Frente
                    
2:
                         140  m² cubie.
                    
2:
                         2 dorm.
                   

In [17]:
descripcion_viv

[['0:\n                         2 dorm.\n                    ',
  '0:\n                         15 años\n                    ',
  '0:\n                         2 baños\n                    ',
  '0:\n                         3 ambientes\n                    ',
  '0:\n                         1 cocheras\n                    ',
  '0:\n                         Excelente\n                    ',
  '0:\n                         Contra.\n                    ',
  '1:\n                         400  m² cubie.\n                    ',
  '1:\n                         5 dorm.\n                    ',
  '1:\n                         30 años\n                    ',
  '1:\n                         3 baños\n                    ',
  '1:\n                         4 ambientes\n                    ',
  '1:\n                         3 cocheras\n                    ',
  '1:\n                         1 toilettes\n                    ',
  '1:\n                         Frente\n                    ',
  '2:\n       

In [38]:
df_faltante =pd.DataFrame({'tipo_moneda':tipomoneda_vivienda,'precio_vivienda':precio_vivienda,
                'direccion_vivienda':direccion_vivienda,'barrio_vivienda':barrio_vivienda,
                'descripcion_vivenda':descripcion_viv,'inmobiliaria':link_inmobiliaria})

In [39]:
df_faltante.to_csv('argenpropfaltante.csv',index=False)